In [1]:
import sys
sys.path.append('..')
import temimbo

import importlib
importlib.reload(temimbo)

<module 'temimbo' from 'c:\\Users\\franz\\OneDrive\\Desktop\\BA\\BA\\LanguageTeacher\\notebooks\\..\\temimbo\\__init__.py'>

In [2]:
tg = temimbo.TaskGenerator(
    openai_endpoint = 'http://I-walk-barefoot.cogsci',
    openai_key = 'abc123',
)
evaluator = temimbo.AnswerEvaluator(
    openai_endpoint = 'https://very-open-but-actually-proprietary.microsoft.com',
    openai_key = 'wow-this-is-secure',
)


ui = temimbo.UserInterface()
db = temimbo.DatabaseClient(connection_string='http://connect-to-my-cerberus:666')
formater = temimbo.Formater()



In [3]:
profile = await db.load_profile(id='Franzy')
domain = await ui.choose_domain()
task_type = await ui.choose_task_type()
level, training_goals_subset = await tg.incorporate_profile_in_task(profile, domain)

In [4]:
prompt = await tg.generate_prompt(
    level = level,
    training_goals_subset = training_goals_subset,
    domain = domain,
    task_type = task_type,
)

raw_output = await tg.generate_task(prompt)

formatted_output_task = await formater.output_task_formatting(raw_output)

In [5]:
user_answer = await ui.answer_task(formatted_output_task)
formated_user_answer = await formater.learner_answer_formatting(user_answer)
NL_feedback, correctness, training_goals = await evaluator.evaluate_learner_answer(
    domain = domain,
    formatted_output_task = formatted_output_task,
    formated_user_answer = formated_user_answer,
    task_type = task_type,
)
print('NL_feedback =', NL_feedback)
print('correctness =', correctness)
print('training_goals =', training_goals)

NL_feedback = You dumbass, can't you see that...
correctness = False
training_goals = vocabulary_goals=['Cat anatomical parts'] grammar_goals=["Use of don't", 'Simple past, even though pasts are never simple'] text_goals=[]


In [6]:
new_profile = await evaluator.update_learner_profile(training_goals, profile)
print(f'Old profile:\n{profile}\n\nNew profile:\n{new_profile}')

Old profile:
name='Franzy' level=Level(vocabulary_level=5, grammar_level=7, text_level=3) training_goals=TrainingGoals(vocabulary_goals=['Foods and drinks'], grammar_goals=[], text_goals=['Describe delicious things'])

New profile:
name='Franzy' level=Level(vocabulary_level=5, grammar_level=7, text_level=3) training_goals=TrainingGoals(vocabulary_goals=['Foods and drinks', 'Cat anatomical parts'], grammar_goals=["Use of don't", 'Simple past, even though pasts are never simple'], text_goals=['Describe delicious things'])


In [7]:
eval_summary = await evaluator.give_feedback(
    NL_feedback,
    correctness,
)
print(eval_summary)

Oh my darling, your mistakes are comprehensible, but...
